In [ ]:
import requests
import pandas as pd

import json

from bs4 import BeautifulSoup

from model import Model

In [ ]:
def fetch_report(report_url: str):
    payload = ""
    headers = {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-US,en;q=0.9",
        "Cache-Control": "max-age=0",
        "Cookie": "bm_mi=2F5AC7DBEC90CA72B7048BFB724142E4~YAAQVCR+aDfBglWNAQAAXdbhXxZRJ4RnkkNmfKWUeKc1QmGH7v3TlPngbOdNdESvidaNFbsxYRDuqNOeHf7o1570sBMa6264ltivVUhPKdRHSGYpVZ2XMWAD/nsmFsSdGcaKYGWMYw5uwfMf/PBPBlZjEQFik5jnd6rWMARaXnpOcYh9njmeVVV519UStSmkmZI5vOvjdoluWej0Z2CxX3Tvw6lNgc1tGVpPsmufVg31H7mrJXYEd2r8dgK/8ByPdjyyHV9vmHGVXWgtIYQEPdFuSZTZcKKd+BnNxBwJMemL5bYVUglFp1bWgOrjeN1N3ix8AELEMXzUgBOh3O3bOm20cMppfRlldfsle3K8Ih4OPhQV7X2gCBWcP/IF9xpVVVUCvbQc36+ZPkQeYwCpp1ZbDy4=~1; bm_sv=6FE42A06849FED4835CF97262309E48E~YAAQVCR+aELBglWNAQAA9N3hXxbgrPGTXP4dLr20mIrEPGqqt/fAg73yGCttn0ty9QTvryHyxJIj5/Xk9bdr1pg+N8DKmsja3McwAOPL66B3PbRpOXDni0I/7hj2onuFL1tySgSQdD5H2wbu2hHzLereVi/obIf/64d3G6DMRmi21FiZaZXw0bKx8Dcbey35f4/a0B3//5sv5dyHH022pza3IR0waWQFv9Up4U5Z1kAbE6dcCBfgw81JPrQ0CA==~1; ak_bmsc=A56127CD072C24354C54D9D888F73150~000000000000000000000000000000~YAAQVCR+aB3CglWNAQAAPE7iXxZxAH8EfBgLMoBK20p82jHscZyuohv2ZDf4igBKSFrl/lk2WxTlHBCZjxlNf0g7+4qhiHtb+iiwCO6dsttHAG4aLtjhcRUqGypjDKB5LdHa2gdTTYL6d+zDand7hgzuYEMTmjSlIXjsdiIpL5cRkkkc62gsj4VYGbKiS7sXf8ZJrpwcTVflEsete1zfSn9uR5h8ydDbzbLPj/YOTNBCNU2lE61l66SPhy3c3LkvFWNB33V1iZVX/r7AqomFCHZQrZiEejEOczLuLEfLyEE/ol0OlL5UgHM/KIfQtfG7xhej2JoGiKzIBRyzQT7iChekHOjlzZgF0IvCURxPvw4bLD14OlSOsh9QYMBkH6MEs8csRoOCbuBdYFKkXESQ+dC0QkyTlDgDbzv61UZ6u/Tzsp3AlkDvWHW2FLNVdZE8G47QP0auueW3sMi84Y3n4reVALhIZUHywXQ6w4F4K3P3fQwcU+F/qOFevEsQU60mNxbPeEc=",
        "If-Modified-Since": "Fri, 22 Feb 2019 21 17:19 GMT",
        "Sec-Ch-Ua": '"Not A(Brand";v="99", "Brave";v="121", "Chromium";v="121"',
        "Sec-Ch-Ua-Mobile": "?0",
        "Sec-Ch-Ua-Platform": '"macOS"',
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Sec-Gpc": "1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
    }

    return requests.request("GET", report_url, data=payload, headers=headers)


def extract_sentences(report) -> list:
    soup = BeautifulSoup(report.text, "xml")
    sentences = []
    for span in soup.find_all("span"):
        text = span.text
        if len(text.split(" ")) > 3:
            sentences.append(text)
    return sentences


def load_sentences(report_url: str) -> list:
    report = fetch_report(report_url)
    return extract_sentences(report)


def analyze_texts(sentences: list, env_pipe, soc_pipe, gov_pipe) -> pd.DataFrame:
    env = env_pipe(sentences, padding=True, truncation=True)
    soc = soc_pipe(sentences, padding=True, truncation=True)
    gov = gov_pipe(sentences, padding=True, truncation=True)

    env_labels = [x["label"] for x in env]
    soc_labels = [x["label"] for x in soc]
    gov_labels = [x["label"] for x in gov]

    return pd.DataFrame(
        {
            "sentence": range(len(sentences)),
            "environmental": env_labels,
            "social": soc_labels,
            "governance": gov_labels,
        }
    )


def load_filings():
    with open("../filings.json", "r") as f:
        filings = json.load(f)
    return filings

In [ ]:
model = Model()
env_pipe = model.load_pipe("env")
soc_pipe = model.load_pipe("soc")
gov_pipe = model.load_pipe("gov")

In [ ]:
filings = load_filings()

In [ ]:
analyses = {}
for ticker, years in filings.items():
    for year in years:
        report_url = filings[ticker][year]
        sentences = load_sentences(report_url)
        analyses[ticker] = {year: analyze_texts(sentences)}